In [1]:
import pandas as pd
import pareto
import os
import sys
from itertools import chain
from matplotlib import pyplot as plt
sys.path.append('..')

In [2]:
import rbf_functions

# Load solutions for each RBF for all seeds:

In [3]:
rbfs = [rbf_functions.squared_exponential_rbf,
        rbf_functions.original_rbf,
        rbf_functions.inverse_quadratic_rbf,
        rbf_functions.inverse_multiquadric_rbf,
        rbf_functions.exponential_rbf,
        rbf_functions.matern32_rbf,
        rbf_functions.matern52_rbf,
       ]

pareto_sets = {}
for entry in rbfs:
    sollist = []
    solutions = []
    name = entry.__name__
    output_dir = f"../output/{name}/"
    for filename in os.listdir(output_dir):
        if filename.endswith('solution.csv'):
            sollist.append(filename[:-4])
            df_temp = pd.read_csv(f"{output_dir}{filename}", header=0)
            solutions.append(df_temp.values.tolist())
    pareto_sets[name] = list(chain.from_iterable(solutions))

pareto_sets.keys()
print("Amount of solutions for each RBF:")
for rbf in pareto_sets:
    print(rbf, len(pareto_sets[rbf]))
# print(f"Loaded into list 'solutions': {', '.join(sollist)}")

Amount of solutions for each RBF:
squared_exponential_rbf 8227
original_rbf 9273
inverse_quadratic_rbf 5385
inverse_multiquadric_rbf 6296
exponential_rbf 4056
matern32_rbf 6266
matern52_rbf 4482


# Create reference set for each RBF, and save:

In [4]:
reference_sets = {}
for rbf in pareto_sets:
    print(rbf, len(pareto_sets[rbf]))
    nondominated = pareto.eps_sort([pareto_sets[rbf]], [0,1,2,3,4,5], [0.5, 0.05, 0.05, 0.05, 0.001, 0.05], maximize=[0,1,2,3,5])
    reference_sets[rbf] = nondominated
    df_nondom = pd.DataFrame(nondominated, columns=['hydropower', 'atomicpowerplant', 'baltimore', 'chester', 'environment', 'recreation'])
    print(rbf, len(df_nondom))
    df_nondom.to_csv(f"../notebooks/refsets/{rbf}_refset.csv", index=False, header=True)

squared_exponential_rbf 8227
squared_exponential_rbf 1357
original_rbf 9273
original_rbf 1514
inverse_quadratic_rbf 5385
inverse_quadratic_rbf 1266
inverse_multiquadric_rbf 6296
inverse_multiquadric_rbf 1157
exponential_rbf 4056
exponential_rbf 946
matern32_rbf 6266
matern32_rbf 1294
matern52_rbf 4482
matern52_rbf 857


# Find decision variables that belong to the generated refset:

In [35]:
d_refvar = {}
d_refsol = {}
for entry in rbfs:
    name = entry.__name__
    df_sol = pd.DataFrame(columns=['hydropower', 'atomicpowerplant', 'baltimore', 'chester', 'environment',
           'recreation'])
    df_var = pd.DataFrame(columns=[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
                 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])
    output_dir = f"../output/{name}"
    for filename in os.listdir(output_dir):
        #load 
        if filename.endswith('solution.csv'):
            df_ts = pd.read_csv(f"{output_dir}/{filename}", header=0)
            df_sol = df_sol.append(df_ts, ignore_index=True)
        elif filename.endswith('variables.csv'):
            df_tv = pd.read_csv(f"{output_dir}/{filename}", header=None)
            df_var = df_var.append(df_tv, ignore_index=True)
    sol = df_sol.values.tolist()
    var = df_var.values.tolist()
    ref = reference_sets[name]
    refvar = []
    refsol = []
    for idx, value in enumerate(sol):
        if value in ref:
            refvar.append(var[idx])
            refsol.append(sol[idx])
    d_refsol[name] = pd.DataFrame(refsol, columns=['hydropower', 'atomicpowerplant', 'baltimore', 'chester', 'environment',
           'recreation'])
    d_refvar[name] = pd.DataFrame(refvar)
    d_refsol[name].to_csv(f"../notebooks/refsets/{name}_refset.csv", index=False, header=True)
    d_refvar[name].to_csv(f"../notebooks/refsets/{name}_refset_variables.csv", index=False, header=False)

In [37]:
for entry in rbfs:
    name = entry.__name__
    print(name)
    print(f"refset: {len(d_refsol[name])}")
    print(f"varset: {len(d_refvar[name])}")

squared_exponential_rbf
refset: 1357
varset: 1357
original_rbf
refset: 1514
varset: 1514
inverse_quadratic_rbf
refset: 1266
varset: 1266
inverse_multiquadric_rbf
refset: 1157
varset: 1157
exponential_rbf
refset: 946
varset: 946
matern32_rbf
refset: 1294
varset: 1294
matern52_rbf
refset: 857
varset: 857


# Generate global reference set for all RBFs:

In [22]:
x = 0
for rbf in pareto_sets:
    x += len(pareto_sets[rbf])
    print(rbf, len(pareto_sets[rbf]))
print("total:", x)
    
pareto_set = {}
sollist = []
solutions = []
for entry in rbfs:
    name = entry.__name__
    output_dir = f"../output/{name}/"
    for filename in os.listdir(output_dir):
        if filename.endswith('solution.csv'):
            sollist.append(filename[:-4])
            df_temp = pd.read_csv(f"{output_dir}{filename}", header=0)
            solutions.append(df_temp.values.tolist())
pareto_set = list(chain.from_iterable(solutions))
len(pareto_set)

squared_exponential_rbf 8227
original_rbf 9273
inverse_quadratic_rbf 5385
inverse_multiquadric_rbf 6296
exponential_rbf 4056
matern32_rbf 6266
matern52_rbf 4482
total: 43985


43985

In [25]:
print(len(pareto_set))
nondominated = pareto.eps_sort([pareto_set], [0,1,2,3,4,5], [0.5, 0.05, 0.05, 0.05, 0.001, 0.05], maximize=[0,1,2,3,5])
df_nondom = pd.DataFrame(nondominated, columns=['hydropower', 'atomicpowerplant', 'baltimore', 'chester', 'environment', 'recreation'])
print(len(nondominated))
df_nondom.to_csv(f"../notebooks/nboutput/all_rbf_refset.csv", index=False, header=True)

43985
1726
